<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/larq1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pasos para Implementar y Comparar VGG8 Normal vs. Hardware-Aware en Larq

**Larq** es una librería diseñada para construir y entrenar redes neuronales binarizadas (BNNs) utilizando TensorFlow y Keras. Si estás interesado en realizar un entrenamiento consciente del hardware (considerando ruido, cuantización, etc.) para **VGG8** usando **Larq**, puedes lograrlo aprovechando las funcionalidades específicas de **Larq** para la binarización y entrenamiento de modelos compactos.

In [2]:
!pip install protobuf==3.20.3

!pip -q install --upgrade larq larq-zoo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
larq-zoo 2.3.2 requires protobuf<3.20, but you have protobuf 3.20.3 which is incompatible.
zookeeper 1.3.4 requires protobuf<3.20, but you have protobuf 3.20.3 which is incompatible.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-aiplatform 1.70.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigquery-connection 1.15.5 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigquery-storage 2.27.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-bigtable 2.26.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,

#### **2. Definición del Modelo VGG8 Normal**
Primero, definimos la arquitectura estándar de **VGG8** en Larq sin binarización: